In [4]:
from problog import get_evaluatable
from problog.program import PrologString, PrologFile

from examples.example2 import *
from examples.time_tests.time_test_recursive_precomp import *
from examples.time_tests.time_test_recursive_end import *
from examples.time_tests.time_test_recursive_iterative_improved import *
from examples.time_tests.time_test_independent_end import *

In [2]:
def save_as_pdf(arithmetic_circuit, filepath):
    dot_format = arithmetic_circuit.to_dot()
    
    with open('/tmp/dot_format.dot', 'w') as f:
        f.write(dot_format)
        
    !dot -Tpdf /tmp/dot_format.dot > $filepath

In [3]:
problog_code = """
0.05 :: burglary.
0.01 :: earthquake.
0.45 :: at_home(john).
0.65 :: at_home(mary).
alarm :- burglary.
alarm :- earthquake.
calls(X) :- at_home(X), alarm.

query(calls(mary))."""

In [33]:
ac = get_evaluatable().create_from(PrologString(problog_code))

In [38]:
save_as_pdf(ac, '/home/marc/thesis/arithmetic_circuits/burglary.pdf')

In [5]:
save_as_pdf(
    get_evaluatable().create_from(PrologFile('/tmp/game.pl')), 
    '/tmp/game.pdf'
)

In [6]:
save_as_pdf(
    get_evaluatable().create_from(PrologFile('/tmp/game2.pl')), 
    '/tmp/game2.pdf'
)

In [7]:
precomp.precompilations

{'atTime': {'model': <problog.sdd_formula.SDD at 0x7efe1f81df28>,
  'nodes': {'0.0::atTime(0).\n': 1,
   '0.0::increase(1).\n': 4,
   '0.0::decrease(1).\n': 2},
  'default_probs': {1: 0.0, 4: 0.0, 2: 0.0},
  'base_timestamp': 1}}

In [3]:
problog_filename = 'examples/example2.pl'

In [15]:
number_of_events = 3

In [21]:
round_probability_to = 6

In [31]:
input_events = get_input_events_recursive(number_of_events)

input_events

{0: [0.7479973023291677::decrease(0).],
 1: [0.14750829804688692::increase(1).],
 2: [0.4612357036676258::decrease(2).]}

In [32]:
for l in input_events.values():
    for e in l:
        e.probability = round(e.probability, round_probability_to)
        
input_events

{0: [0.747997::decrease(0).],
 1: [0.147508::increase(1).],
 2: [0.461236::decrease(2).]}

In [43]:
for l in input_events.values():
    for e in l:
        print(str(e).strip())

0.747997::decrease(0).
0.147508::increase(1).
0.461236::decrease(2).


In [33]:
queries = get_query_recursive(number_of_events)

queries

{0: <examples.example2.FeedbackQuery at 0x7fc84f508f60>,
 1: <examples.example2.FeedbackQuery at 0x7fc84f508f28>,
 2: <examples.example2.FeedbackQuery at 0x7fc84f508dd8>}

In [34]:
precomp = generate_feedback_precompilation(problog_filename)

save_as_pdf(
    precomp.precompilations['atTime']['model'], '/home/marc/thesis/arithmetic_circuits/recursive_precomp.pdf'
)

In [35]:
with open(problog_filename, 'r') as f:
    problog_code = ''.join([l for l in f])

all_inputs = join_list_of_lists_to_problog(input_events.values())
all_queries = join_list_of_lists_to_problog(
    {
        i: [q]
        for i, q in queries.items()
    }.values()
)

prolog_string = PrologString(
    '\n'.join([problog_code, all_inputs, all_queries])
)

save_as_pdf(
    get_evaluatable().create_from(prolog_string), '/home/marc/thesis/arithmetic_circuits/recursive_at_end.pdf'
)

In [36]:
get_evaluatable().create_from(prolog_string).evaluate()

{atTime(0): 0.0, atTime(1): 0.147508, atTime(2): 0.07947200011200001}

In [37]:
with open('examples/example2_iter_imp.pl', 'r') as f:
    problog_imp_code = ''.join([l for l in f])

    
res = {}
feedback = None
for i in range(number_of_events):
    query = queries[i]
    inputs = list(input_events[i])
    if feedback:
        inputs += feedback
        
    inputs_str = '\n'.join(map(lambda x: x.to_problog(), inputs))
    queries_str = '\n'.join(map(lambda x: x.to_problog(), [query]))

    prolog_string = PrologString(
        '\n'.join([problog_imp_code, inputs_str, queries_str])
    )

    ac = get_evaluatable().create_from(prolog_string)
    
    save_as_pdf(
        ac, '/home/marc/thesis/arithmetic_circuits/iterative_imp_{}.pdf'.format(i)
    )

    current_res = ac.evaluate()

    feedback = my_generate_feedback(current_res)

    res.update(current_res)

In [38]:
res

{atTime(0): 0.0, atTime(1): 0.147508, atTime(2): 0.07947200011200001}